# Импорт библиотек

In [5]:
import pandas as pd
import numpy as np
import zipfile as zf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import PredefinedSplit

from tqdm.notebook import tqdm

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pickle

from sklearn.metrics import f1_score

# Загрузка и очистка данныx

In [6]:
# file = zf.ZipFile('/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/data_4class.zip')
# file.extractall()

In [7]:
df = pd.read_csv(r'C:\Users\ffedo\Desktop\data_science\data\data_4class\data_4class.csv', sep=';')
df

,x1,x2,x3,x4,x5,x6,class
0,"4,515297646252625","-2,07036706615714","3,2585383730838378","3,7930594045972192","3,4720484621322356","-0,4789201770813123",3
1,"1,5508794670649269","1,3820146712061634","1,942693485886148","2,069885683865084","-0,22511193683695682","-2,303949682856058",2
2,"2,9880972406280066","-2,1365756342215176","2,469954527100078","1,7252063508758542","2,971663586773672","-2,543173242953806",3
3,"3,139767963592046","-1,677145546231046","4,138218458647584","2,521013012549771","2,650713586139205","-0,3626219014045815",3
4,"3,1197797829928464","-2,10074721921465","3,699075357830096","2,0461730318657594","2,9990916631660087","-1,6752961527892165",3
...,...,...,...,...,...,...,...
79995,"1,4322293723901158","2,577585066292325","2,9289348771877126","2,469154608484738","-1,0892062411168344","-3,0116797224659697",3
79996,"5,542870566778231","-4,350409600487184","0,3511813929470158","2,6238820108051475","5,749305074824563","-6,686529946039072",3
79997,"2,63199942217274","-1,7480177028185402","3,0505605609724165","1,7208149055436084","2,514794267502953","-1,518916856679832",3
79998,"3,982406251164055","2,620576354922133","-2,7709626535506384","5,519990383556996",NaN,"-1,3776815330357355",2


In [8]:
df.duplicated().sum() # проверка на дубликаты

0

In [9]:
df.isna().sum().sum() # проверкаа на пропуски

27158

In [10]:
df = df.dropna() # удаление пропусков
print(df.isna().sum().sum())
df

0


,x1,x2,x3,x4,x5,x6,class
0,"4,515297646252625","-2,07036706615714","3,2585383730838378","3,7930594045972192","3,4720484621322356","-0,4789201770813123",3
1,"1,5508794670649269","1,3820146712061634","1,942693485886148","2,069885683865084","-0,22511193683695682","-2,303949682856058",2
2,"2,9880972406280066","-2,1365756342215176","2,469954527100078","1,7252063508758542","2,971663586773672","-2,543173242953806",3
3,"3,139767963592046","-1,677145546231046","4,138218458647584","2,521013012549771","2,650713586139205","-0,3626219014045815",3
4,"3,1197797829928464","-2,10074721921465","3,699075357830096","2,0461730318657594","2,9990916631660087","-1,6752961527892165",3
...,...,...,...,...,...,...,...
79994,"-3,6889522034691513","-2,03030768541395","5,561773755122672","-5,125655762226804","0,19355716473086346","-0,20490687574481736",0
79995,"1,4322293723901158","2,577585066292325","2,9289348771877126","2,469154608484738","-1,0892062411168344","-3,0116797224659697",3
79996,"5,542870566778231","-4,350409600487184","0,3511813929470158","2,6238820108051475","5,749305074824563","-6,686529946039072",3
79997,"2,63199942217274","-1,7480177028185402","3,0505605609724165","1,7208149055436084","2,514794267502953","-1,518916856679832",3


# Предобработка данных и выделение значимых атрибутов

In [11]:
df = df.reindex(range(len(df)), method='ffill')
df

,x1,x2,x3,x4,x5,x6,class
0,"4,515297646252625","-2,07036706615714","3,2585383730838378","3,7930594045972192","3,4720484621322356","-0,4789201770813123",3
1,"1,5508794670649269","1,3820146712061634","1,942693485886148","2,069885683865084","-0,22511193683695682","-2,303949682856058",2
2,"2,9880972406280066","-2,1365756342215176","2,469954527100078","1,7252063508758542","2,971663586773672","-2,543173242953806",3
3,"3,139767963592046","-1,677145546231046","4,138218458647584","2,521013012549771","2,650713586139205","-0,3626219014045815",3
4,"3,1197797829928464","-2,10074721921465","3,699075357830096","2,0461730318657594","2,9990916631660087","-1,6752961527892165",3
...,...,...,...,...,...,...,...
56427,"1,654091590850613","2,3052018415206277","2,9783290888204585","2,6311445602646035","-0,8146397462591817","-2,633101200590537",0
56428,"1,654091590850613","2,3052018415206277","2,9783290888204585","2,6311445602646035","-0,8146397462591817","-2,633101200590537",0
56429,"5,22961597904812","-2,507693395962547","2,794338386098837","4,217209458413095","4,099407546039043","-1,1976450237215426",3
56430,"4,556475532469856","-3,417744660051489","1,5635614553796477","2,427510189563869","4,604257637400534","-4,463168098990028",1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56432 entries, 0 to 56431
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x1      56432 non-null  object
 1   x2      56432 non-null  object
 2   x3      56432 non-null  object
 3   x4      56432 non-null  object
 4   x5      56432 non-null  object
 5   x6      56432 non-null  object
 6   class   56432 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 3.0+ MB


In [13]:
# подготовка к переводу во float
df.x1 = df.x1.str.replace(",", ".")
df.x2 = df.x2.str.replace(",", ".")
df.x3 = df.x3.str.replace(",", ".")
df.x4 = df.x4.str.replace(",", ".")
df.x5 = df.x5.str.replace(",", ".")
df.x6 = df.x6.str.replace(",", ".")
df

,x1,x2,x3,x4,x5,x6,class
0,4.515297646252625,-2.07036706615714,3.2585383730838378,3.7930594045972192,3.4720484621322356,-0.4789201770813123,3
1,1.5508794670649269,1.3820146712061634,1.942693485886148,2.069885683865084,-0.22511193683695682,-2.303949682856058,2
2,2.9880972406280066,-2.1365756342215176,2.469954527100078,1.7252063508758542,2.971663586773672,-2.543173242953806,3
3,3.139767963592046,-1.677145546231046,4.138218458647584,2.521013012549771,2.650713586139205,-0.3626219014045815,3
4,3.1197797829928464,-2.10074721921465,3.699075357830096,2.0461730318657594,2.9990916631660087,-1.6752961527892165,3
...,...,...,...,...,...,...,...
56427,1.654091590850613,2.3052018415206277,2.9783290888204585,2.6311445602646035,-0.8146397462591817,-2.633101200590537,0
56428,1.654091590850613,2.3052018415206277,2.9783290888204585,2.6311445602646035,-0.8146397462591817,-2.633101200590537,0
56429,5.22961597904812,-2.507693395962547,2.794338386098837,4.217209458413095,4.099407546039043,-1.1976450237215426,3
56430,4.556475532469856,-3.417744660051489,1.5635614553796477,2.427510189563869,4.604257637400534,-4.463168098990028,1


In [14]:
df = df.astype(float) # перевод во float
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56432 entries, 0 to 56431
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      56432 non-null  float64
 1   x2      56432 non-null  float64
 2   x3      56432 non-null  float64
 3   x4      56432 non-null  float64
 4   x5      56432 non-null  float64
 5   x6      56432 non-null  float64
 6   class   56432 non-null  float64
dtypes: float64(7)
memory usage: 3.0 MB


In [15]:
X = df.drop(['class'], axis=1) # выбор признаков
X

,x1,x2,x3,x4,x5,x6
0,4.515298,-2.070367,3.258538,3.793059,3.472048,-0.478920
1,1.550879,1.382015,1.942693,2.069886,-0.225112,-2.303950
2,2.988097,-2.136576,2.469955,1.725206,2.971664,-2.543173
3,3.139768,-1.677146,4.138218,2.521013,2.650714,-0.362622
4,3.119780,-2.100747,3.699075,2.046173,2.999092,-1.675296
...,...,...,...,...,...,...
56427,1.654092,2.305202,2.978329,2.631145,-0.814640,-2.633101
56428,1.654092,2.305202,2.978329,2.631145,-0.814640,-2.633101
56429,5.229616,-2.507693,2.794338,4.217209,4.099408,-1.197645
56430,4.556476,-3.417745,1.563561,2.427510,4.604258,-4.463168


In [16]:
y = df['class'].astype(int) # выбор целевого признака
y

0        3
1        2
2        3
3        3
4        3
        ..
56427    0
56428    0
56429    3
56430    1
56431    1
Name: class, Length: 56432, dtype: int32

# Подготовка обучающей и проверочной выборок

In [17]:
# нормализация данных
scaler = StandardScaler()
X_ss = scaler.fit_transform(X)
X_ss.shape

(56432, 6)

In [18]:
# добавление полиномиальных фич
poly = PolynomialFeatures(2)
X_ss_poly = poly.fit_transform(X_ss)

print(X_ss_poly.shape)

(56432, 28)


In [19]:
# X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
# X_train, X_test, y_train, y_test = train_test_split(X_ss_poly, y, test_size=0.1, stratify=y)

# Сравнение различных моделей машинного обучения

## Деревo

In [20]:
dtc = DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)

In [21]:
# pickle.dump(dtc, "/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/dtc.pkl")

In [22]:
# dtc = pickle.load("/content/drive/MyDrive/data science/ML/Самостоятельная работа ML/dtc.pkl")

In [23]:
dtc_pred = dtc.predict(X_test)
dtc_pred

array([0, 0, 0, ..., 0, 1, 0])

In [24]:
f1_score(y_test, dtc.predict(X_test), average="micro")

0.8825301204819277

## Случайный лес

In [25]:
rfc = RandomForestClassifier()

In [26]:
# rfc = rfc.fit(X_train, y_train)
# f1_score(y_test, rfc.predict(X_test), average="macro")

In [27]:
# визуализация валидационной выборки для обучения
ps = PredefinedSplit(test_fold=[-1 if i in X_train.index else 0 for i in X.index])
ps

PredefinedSplit(test_fold=array([-1, -1, ..., -1, -1]))

In [30]:
rfc_rs = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid={
        'n_estimators': [20, 50, 100, 200, 250, 300],
        'max_depth': [7, 8, 10, 20, 25, None],
        'max_features': ['sqrt', 'log2', None],
        'criterion': ['gini', 'entropy', 'log_loss'],
        'min_samples_split': [1, 2, 5, 10],
        'n_jobs': [-1]
    },
    scoring='f1_micro',
    verbose=52,
    cv=ps
)
# model.fit(X_train, y_train)
rfc_rs.fit(X, y)
f1_score(y_test, rfc_rs.predict(X_test), average='micro')

Fitting 1 folds for each of 1296 candidates, totalling 1296 fits
[CV 1/1; 1/1296] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=20, n_jobs=-1
[CV 1/1; 1/1296] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=20, n_jobs=-1;, score=nan total time=   0.0s
[CV 1/1; 2/1296] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=50, n_jobs=-1
[CV 1/1; 2/1296] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=50, n_jobs=-1;, score=nan total time=   0.0s
[CV 1/1; 3/1296] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=100, n_jobs=-1
[CV 1/1; 3/1296] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=100, n_jobs=-1;, score=nan total time=   0.0s
[CV 1/1; 4/1296] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=200, n_jobs=-1
[CV 1/1; 4/1

c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
324 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
324 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
  File "c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
  File "c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
  File "c:\Users\ffedo\AppData\Local\Programs\

1.0

In [ ]:
# pickle.load(r'C:\Users\ffedo\Desktop\data_science\models\rfc_rs.pkl')

In [31]:
f1_score(y_test, rfc_rs.predict(X_test), average="micro")

1.0

In [32]:
rfc_rs.best_params_

{'criterion': 'gini',
 'max_depth': None,
 'max_features': 'log2',
 'min_samples_split': 2,
 'n_estimators': 200,
 'n_jobs': -1}

In [33]:
rfc_rs_df = pd.DataFrame(rfc_rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rfc_rs_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators,param_n_jobs,params,split0_test_score,mean_test_score,std_test_score,rank_test_score
0,4.168445,0.0,0.049760,0.0,gini,None,log2,2,200,-1,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.937987,0.937987,0.0,1
1,5.167902,0.0,0.051132,0.0,gini,None,sqrt,2,250,-1,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.937810,0.937810,0.0,2
2,4.187043,0.0,0.051553,0.0,gini,None,sqrt,2,200,-1,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.937810,0.937810,0.0,2
3,7.568704,0.0,0.051955,0.0,log_loss,None,sqrt,2,250,-1,"{'criterion': 'log_loss', 'max_depth': None, '...",0.937633,0.937633,0.0,4
4,6.179810,0.0,0.061705,0.0,gini,None,sqrt,2,300,-1,"{'criterion': 'gini', 'max_depth': None, 'max_...",0.937633,0.937633,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,0.002899,0.0,0.000000,0.0,log_loss,8,None,1,100,-1,"{'criterion': 'log_loss', 'max_depth': 8, 'max...",NaN,NaN,NaN,973
1292,0.002001,0.0,0.000000,0.0,log_loss,8,None,1,50,-1,"{'criterion': 'log_loss', 'max_depth': 8, 'max...",NaN,NaN,NaN,973
1293,0.001000,0.0,0.000000,0.0,log_loss,8,None,1,20,-1,"{'criterion': 'log_loss', 'max_depth': 8, 'max...",NaN,NaN,NaN,973
1294,0.001998,0.0,0.000000,0.0,gini,20,None,1,50,-1,"{'criterion': 'gini', 'max_depth': 20, 'max_fe...",NaN,NaN,NaN,973


## Машины опорных векторов

In [41]:
svc = SVC() # lin 0,8871 pol 0,8825 rbf 0,8869 sig 0,7654
# svc = svc.fit(X_train,y_train)
# f1_score(y_test, svc.predict(X_test), average="micro")

In [42]:
parameters = {'C': [15, 20, 25],
              'gamma': ['auto','scale'],
              'kernel': ['linear']}

In [43]:
svc_gs=GridSearchCV(estimator = svc,
                        param_grid = parameters,
                        scoring='f1_micro',
                        verbose=52,
                        cv=2)

svc_gs.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV 1/2; 1/6] START C=15, gamma=auto, kernel=linear.............................
[CV 1/2; 1/6] END C=15, gamma=auto, kernel=linear;, score=0.886 total time= 1.3min
[CV 2/2; 1/6] START C=15, gamma=auto, kernel=linear.............................
[CV 2/2; 1/6] END C=15, gamma=auto, kernel=linear;, score=0.879 total time= 1.1min
[CV 1/2; 2/6] START C=15, gamma=scale, kernel=linear............................
[CV 1/2; 2/6] END C=15, gamma=scale, kernel=linear;, score=0.886 total time= 1.3min
[CV 2/2; 2/6] START C=15, gamma=scale, kernel=linear............................
[CV 2/2; 2/6] END C=15, gamma=scale, kernel=linear;, score=0.879 total time= 1.2min
[CV 1/2; 3/6] START C=20, gamma=auto, kernel=linear.............................
[CV 1/2; 3/6] END C=20, gamma=auto, kernel=linear;, score=0.886 total time= 1.7min
[CV 2/2; 3/6] START C=20, gamma=auto, kernel=linear.............................
[CV 2/2; 3/6] END C=20, gamma=auto, k

ModuleNotFoundError: No module named 'sklearn.utils._pprint'

ModuleNotFoundError: No module named 'sklearn.utils._pprint'

ModuleNotFoundError: No module named 'sklearn.utils._pprint'

In [44]:
f1_score(y_test, svc_gs.predict(X_test), average="micro")

0.881467044649185

In [45]:
svc_gs.best_params_

{'C': 20, 'gamma': 'auto', 'kernel': 'linear'}

In [46]:
svc_gs_df = pd.DataFrame(svc_gs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
svc_gs_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,92.510679,9.022429,2.503926,0.042006,20,auto,linear,"{'C': 20, 'gamma': 'auto', 'kernel': 'linear'}",0.885721,0.879184,0.882453,0.003268,1
1,90.493165,8.200599,2.465336,0.098405,20,scale,linear,"{'C': 20, 'gamma': 'scale', 'kernel': 'linear'}",0.885721,0.879184,0.882453,0.003268,1
2,103.681196,4.092189,2.502295,0.041795,25,auto,linear,"{'C': 25, 'gamma': 'auto', 'kernel': 'linear'}",0.885721,0.879184,0.882453,0.003268,1
3,103.796231,3.898011,2.577197,0.014613,25,scale,linear,"{'C': 25, 'gamma': 'scale', 'kernel': 'linear'}",0.885721,0.879184,0.882453,0.003268,1
4,70.836779,4.245466,2.546774,0.053177,15,auto,linear,"{'C': 15, 'gamma': 'auto', 'kernel': 'linear'}",0.885682,0.879184,0.882433,0.003249,5
5,71.779167,3.897593,2.535468,0.064158,15,scale,linear,"{'C': 15, 'gamma': 'scale', 'kernel': 'linear'}",0.885682,0.879184,0.882433,0.003249,5


# Выбор лучшей модели и ее оценка

In [47]:
# derevo
f1_score(y_test, dtc.predict(X_test), average="micro")

0.8825301204819277

In [48]:
# random forest
f1_score(y_test, rfc_rs.predict(X_test), average="micro")

1.0

In [49]:
# vector
f1_score(y_test, svc_gs.predict(X_test), average="micro")

0.881467044649185

Random Forest показал себя лучше остальных, будем использовать его

In [37]:
# сохраняем лучшую модель в файл
with open(r'C:\Users\ffedo\Desktop\data_science\models\rfc_rs.pkl', 'wb') as file:
    pickle.dump(rfc_rs, file)